In [1]:
from __future__ import annotations

In [ ]:
import math

import ipywidgets as widgets
import numpy as np
import rerun as rr
from rerun.notebook import Viewer, ViewerCallbacks
from rerun.utilities import build_color_grid

In [ ]:
# Log some data
rr.init("rerun_example_callbacks")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))


# Use callbacks to track the current time and active selection
class Example(ViewerCallbacks):
    def __init__(self) -> None:
        self.selection = ""
        self.time = ""
        self.timeline = ""

        self.viewer = Viewer()
        self.viewer.register_callbacks(self)

        self.time_label = widgets.Label()
        self.selection_label = widgets.Label()
        self.label = widgets.VBox([self.time_label, self.selection_label])

    def _ipython_display_(self):
        self.viewer.display()
        display(self.label)

    def update_label(self) -> None:
        self.time_label.value = f"time={self.timeline} @ {self.time}"
        self.selection_label.value = f"selected=[{self.selection}]"

    def on_selection_change(self, selection) -> None:
        label = []
        for item in selection:
            if item.kind == "entity":
                label.append(f"Entity({item.entity_path})")
            elif item.kind == "instance":
                label.append(f"Entity({item.entity_path}[{item.instance_id}])")
            elif item.kind == "container":
                label.append(f"Container({item.container_id})")
            elif item.kind == "view":
                label.append(f"View({item.view_id})")
        self.selection = ", ".join(label)
        self.update_label()

    def on_time_update(self, time) -> None:
        self.time = str(time)
        self.update_label()

    def on_timeline_change(self, timeline, time) -> None:
        self.timeline = str(timeline)
        self.time = str(time)
        self.update_label()


# Start the viewer instance
Example()